In [1]:
import pandas as pd
import sys
sys.path.append('./app')
from ai4bharat.transliteration import XlitEngine
import datasets
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from tqdm import tqdm
import json

now = time.time()
dataset = datasets.load_from_disk('../punjabi/CulturaX_tokenized')

tranliterated_words_map = {}
with open('tranliterated_words_map.json', 'r') as f:
    tranliterated_words_map = json.load(f)
    
dataset




/home/kd/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text', 'timestamp', 'url', 'source', 'input_ids', 'attention_mask', 'input_ids_len'],
        num_rows: 646987
    })
})

In [2]:
dataset = dataset['train']
e = XlitEngine(beam_width=4, src_script_type='indic', lang2use='pa', batch_size = 256)
def transliterate_pa(example):
    return {'transliterated_text': e.translit_sentence(example['text'], 'pa')}

Initializing Multilingual model for transliteration


/home/kd/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Loading dicts into RAM: 100%|██████████| 1/1 [00:00<00:00, 24.65it/s]


In [5]:
transliterated_dataset = dataset.map(transliterate_pa, batched=False, num_proc=1)
transliterated_dataset


Map: 100%|██████████| 646987/646987 [23:50<00:00, 452.19 examples/s]  


Dataset({
    features: ['text', 'timestamp', 'url', 'source', 'input_ids', 'attention_mask', 'input_ids_len', 'transliterated_text'],
    num_rows: 646987
})

In [12]:
for i in range(23):
    print(transliterated_dataset['text'][i], end='\n')
    print(transliterated_dataset['transliterated_text'][i], end='\n\n\n')

ਅਜੀਤ : ਫਿਰੋਜ਼ਪੁਰ / ਫਾਜ਼ਿਲਕਾ / ਅਬੋਹਰ - ਕਾਨੂੰਨ ਰੱਦ ਕਰਨ ਦਾ ਫ਼ੈਸਲਾ ਸ਼ਲਾਘਾਯੋਗ-ਟੁਟੇਜਾ
ਮੰਡੀ ਅਰਨੀਵਾਲਾ, 24 ਨਵੰਬਰ (ਨਿਸ਼ਾਨ ਸਿੰਘ ਸੰਧੂ)-ਸੀਨੀਅਰ ਕਾਂਗਰਸੀ ਆਗੂ ਰਮੇਸ਼ ਟੁਟੇਜਾ ਅਰਨੀਵਾਲਾ ਨੇ ਪ੍ਰਧਾਨ ਮੰਤਰੀ ਨਰਿੰਦਰ ਮੋਦੀ ਵਲੋਂ ਤਿੰਨ ਖੇਤੀ ਕਾਨੂੰਨ ਰੱਦ ਕੀਤੇ ਜਾਣ ਦੇ ਫ਼ੈਸਲੇ ਦਾ ਸਵਾਗਤ ਕਰਦਿਆਂ ਇਸ ਨੂੰ ਦੇਰ ਨਾਲ ਆਇਆ ਫ਼ੈਸਲਾ ਦੱਸਿਆ ਹੈ | ਉਨ੍ਹਾਂ ਕਿਹਾ ਕਿ ਮੁੱਖ ਮੰਤਰੀ ਚਰਨਜੀਤ ਸਿੰਘ ਚੰਨੀ ਦੀ ਅਗਵਾਈ ਹੇਠ ਕਾਂਗਰਸ ਪਾਰਟੀ ਇਨ੍ਹਾਂ ਕਾਨੂੰਨਾਂ ਨੂੰ ਪਹਿਲਾ ਹੀ ਰੱਦ ਕਰ ਚੁੱਕੀ ਹੈ |
ajith : ferozepur / fazilka / abohar - kanoon radd curran dae faisla shalaghayog-tuteja
mandi arniwala, 24 november (nishan singh sandhu)-senior kangarsi aagoo ramesh tuteja arniwala nay pradhan mantri narinder modi valon tinn kheti kanoon radd kite jaanh dey faisle dae swagat kardiaan ish noon der naal aaiaa faisla dassiaa haii | unhan kiha kii mukkh mantri charanjit singh channi dee agvai heth congress party inhan kanoonan noon pahila hee radd qarr chukki haii |


ਪੰਜਾਬ ਦੇ ਵਿਦਿਆਰਥੀਆਂ ਲਈ ਵਰਦਾਨ ਸਾਬਿਤ ਹੋ ਰਿਹੈ 'ਉਡਾਣ' ਪ੍ਰਾਜੈਕਟ
Updated: 14 Jan, 2021 03:27 PM
ਚੰਡੀਗੜ੍ਹ : ਪੰਜਾਬ ਸਕੂਲ ਸਿੱਖਿਆ ਮਹਿਕ

In [13]:
transliterated_dataset.save_to_disk('../punjabi/CulturaX_tokenized')

Saving the dataset (22/22 shards): 100%|██████████| 646987/646987 [00:16<00:00, 40050.81 examples/s]


In [ ]:
unique_words = set()

# read unique words file
with open('unique_words.txt', 'r') as f:
    unique_words.update(f.read().split())

# for entry in tqdm(dataset):
#     words = entry['text'].split()
#     unique_words.update(words)

print(len(unique_words))


In [ ]:
len(unique_words), len(tranliterated_words_map)

In [ ]:
unique_words -= set(tranliterated_words_map.keys())
unique_words = list(unique_words)
unique_words_str = ' '.join(unique_words)

len(unique_words)


In [ ]:
# # save str to file
# with open('unique_words.txt', 'w') as f:
#     f.write(unique_words_str)

In [ ]:
e = XlitEngine(beam_width=4, src_script_type='indic', lang2use='pa', batch_size = 256)


results = []

# split unique words list into chunks of 3000
chunks = [unique_words[x:x+3000] for x in range(0, len(unique_words), 3000)]

for i, chunk in tqdm(enumerate(chunks), total=len(chunks)):
    print(f'Processing chunk {i}')
    chunk_str = ' '.join(chunk)
    result = e.translit_sentence(chunk_str, 'pa')
    results.append(result)
    
# result = e.translit_sentence(unique_words_str, 'pa')


In [ ]:
len(results)

In [ ]:
len(tranliterated_words_map)

In [ ]:
for z in results:
    for k, v in z:
        tranliterated_words_map[k] = v

len(tranliterated_words_map)


In [ ]:
import json

with open('transliterated_words_map.json', 'w') as f:
    json.dump(tranliterated_words_map, f, indent=4)